In [29]:
#  TODO Add google translate API
#  TODO code document translate from pdf with file upload
#  TODO add streaming to responses
#  TODO format key word output for translator to use dictionary
#  TODO add transcription grammar correction for students
#  TODO add google translate api for key word translation
#  TODO make basic backend and front end site to frame connections

In [61]:
message = b'hell'

In [1]:
import gradio as gr
import whisper
import os
from dotenv import load_dotenv
import openai
import logging


load_dotenv(".env")
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [2]:
# Create a logger
logging.basicConfig(filename='example2.log',level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# Generic Chatgpt chat completition with input arguments for system message and user message

def chatbot_completition(system_message, user_message):
    response = openai.ChatCompletion.create(
        model = 'gpt-4',
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message},
        ],
    )
    
    prompt, reply = response['usage']['prompt_tokens'], response['usage']['completion_tokens']
    cost = (0.03/1000)*prompt + (0.06/1000)*reply
    ## calculate api costs and log it
    logger.info(f"Prompt tokens: {prompt}, Completion tokens: {reply}, Cost: {cost}")

    return response["choices"][0]["message"]["content"]

In [4]:
# Translation chain
class Translation:
    def __init__(self, source_language, target_language):
        self.source_language = source_language
        self.target_language = target_language
        
    def translate(self, text):
        logger.info("Starting translation...")
        
        logger.info("Extracting context...")
        context = self.extract_context(text)
        logger.info(f"Context extracted: {context}")
        
        logger.info("Extracting key words...")
        key_words = self.extract_key_words(text)
        logger.info(f"Key words extracted: {key_words}")
        
        SYSTEM = f"""You are an expert {self.target_language} translator. Your role is to carefully examine the given text, which may be written in {self.source_language}, and translate it accurately into {self.target_language}. Your translation work should account for not only literal translation but also cultural and contextual nuances. You'll be given text that has to be translated into {self.target_language}, the context of the text and key words with possible translations. Your task is to ensure the translated text preserves the original message while being grammatically correct and contextually appropriate in {self.target_language}. If the text is not in {self.source_language} you will say "Please enter text in {self.source_language} to translate" Respond using markdown."""
        
        example_en_to_gr = f"""                    
                    "Given this text to translate: "It's a beautiful sunny day at the beach."
                    Translate it from English to Greek:
                    Given the context: "This sentence is describing a pleasant weather condition at a seaside location." and key words: "beautiful (όμορφος), sunny (ηλιόλουστος), day (μέρα), beach (παραλία)"
                    The translation is: "Είναι μια όμορφη ηλιόλουστη μέρα στην παραλία." 
                    """
        example_gr_to_en = f"""
                    "Given this text to translate: "Είναι μια όμορφη ηλιόλουστη μέρα στην παραλία."
                    Translate it from Greek to English:
                    Given the context: "This sentence is describing a pleasant weather condition at a seaside location." and key words: "όμορφη (beautiful), ηλιόλουστη (sunny), μέρα (day), παραλία (beach)"
                    The translation is: "It's a beautiful sunny day at the beach." " 
                    """
                    
        if self.source_language == "English":
            example = example_en_to_gr
        else:
            example = example_gr_to_en
                
        PROMPT = f"""
                    Translate from {self.source_language} to {self.target_language}:
                    
                    Example:
                    {example}
                    
                    Now, given this text to translate: "{text}"
       
                    Given the context: "{context}" and key words: "{key_words}"
                    
                    The translation is:
                    """
        logger.info("Translating text...")
        translation = chatbot_completition(SYSTEM, PROMPT)
        logger.info(f"Translation completed: {translation}")
        
        return translation, key_words
        
    
    def extract_context(self, text):
        '''Extracts context from text using chatgpt api call'''
        
        SYSTEM = "You are a sophisticated AI model capable of understanding and interpreting English text. Your task is to identify and describe the overall context or main topic of the following text."
        PROMPT = f"Please identify the context of this text: \"{text}\""
        context = chatbot_completition(SYSTEM, PROMPT)
        
        return context

    def extract_key_words(self, text):
        '''Extracts key words from text using chatgpt api call'''
        
        if self.source_language == "English":
            SYSTEM = """You are a sophisticated AI model trained to identify key words and phrases in English text. Your task is to extract the most important words or phrases that capture the main points of the following text.
            Example:
            Text:"It's a beautiful sunny day at the beach."
            key words: "beautiful (όμορφος), sunny (ηλιόλουστος), day (μέρα), beach (παραλία)"
            """
        else:
            SYSTEM = """You are a sophisticated AI model trained to identify key words and phrases in Greek text. Your task is to extract the most important words or phrases that capture the main points of the following text.
            Example:
            Text:"Είναι μια όμορφη ηλιόλουστη μέρα στην παραλία."
            Key words: "όμορφη (beautiful), ηλιόλουστη (sunny), μέρα (day), παραλία (beach)"
            """
        PROMPT = f"""Please extract the key words from this text: \"{text}\ and their possible translations in {self.target_language}.
        """
        key_words = chatbot_completition(SYSTEM, PROMPT)
        return key_words


In [5]:
class Transcription:
    def __init__(self, source_language, target_language):
        self.source_language = source_language
        self.target_language = target_language
        self.model = whisper.load_model("large")
        self.translate = Translation(source_language, target_language)

    def transcribe(self, audio_file):
        '''Transcribes the audio file and proofreads the transcription'''

        ### Size (in bytes) = (Sample Rate) * (Bit Depth/8) * (Number of Channels) * (Duration in seconds)
        ### Size = 44,100 * (16/8) * 1 * 60 = 5,292,000 bytes, which is approximately 5 MB.

        # Calculate the size of the audio file
        audio_file_size = os.path.getsize(audio_file)
        audio_length = audio_file_size / ( 44100 * (16/8) * 1 )
        logger.info(f"Audio file size: {audio_file_size} bytes and {audio_length} seconds")

        # Transcribe the audio
        logger.info("Starting transcription...")
        transcription = self.transcribe_audio(audio_file)
        logger.info(f"Transcription completed: {transcription}")

        # Proofread the transcription
        logger.info("Proofreading transcription...")
        proofread_transcription = self.proofread_transcription(transcription)
        logger.info(f"Proofread transcription: {proofread_transcription}")

        # Translate the transcription
        translation,key_words = self.translate.translate(proofread_transcription)
        
        return proofread_transcription, translation, key_words

    def transcribe_audio(self, audio_path):
        '''Transcribes an audio file using the whisper model'''
        
        # Check if file size is less than 25 MB
        file_size = os.path.getsize(audio_path) / (1024 * 1024) # size in MB
        if file_size > 25:
            raise ValueError(f"The file size is {file_size:.2f} MB which exceeds the 25 MB limit.")

        # Check if file extension is supported
        supported_file_types = ['mp3', 'mp4', 'mpeg', 'mpga', 'm4a', 'wav', 'webm']
        file_extension = os.path.splitext(audio_path)[1][1:]
        if file_extension not in supported_file_types:
            raise ValueError(f"The file type '{file_extension}' is not supported. Please use one of the following: {', '.join(supported_file_types)}.")

        # If checks pass, transcribe the audio
        audio_file = open(audio_path, "rb")
        transcript = openai.Audio.transcribe("whisper-1", audio_file)

        # Always close the file you opened
        audio_file.close()

        return transcript
        
        # result = self.model.transcribe(audio_file)        
        # return result["text"]

    def proofread_transcription(self, transcription):
        '''Improves the transcription quality using GPT-4'''
        
        SYSTEM = """You are an expert {self.source_langauge} transcription agent. In particular, an advanced language model trained to proofread and correct text based on grammar, syntax, and coherence in context. Your role is to carefully examine the given text in {self.source_langauge}, identify semantic errors resulting from transcription, and then correct these errors. Use your knowledge of {self.source_language} language and syntax to ensure the accuracy and coherence of the transcriptions. Your task is crucial for the quality and usefulness of the transcriptions. You will be given {self.source_langauge} text that has been transcribed but may contain errors. Your job is to fix these errors to the best of your ability. Respond using markdown.
        
        You will work through proofreading and correcting the transcription in the following way step by step:
        
        Identify the context: [context]
        Identify key words: [key words]
        Correct the transcription: [transcription]
        
        You will not add any additional detail to the transcription. You will only correct the transcription. You will only return the transcription. You will not return the context or key words.
        
        Write the transcription here:
               
        
        """
        
        PROMPT = f"Please proofread and correct this transcription: \"{transcription}\""
        proofread_transcription = chatbot_completition(SYSTEM, PROMPT)
        return proofread_transcription

In [6]:
from PyPDF2 import PdfReader
from docx import Document

class DocumentTranslation(Translation):
    '''Class to translate pdf documents into English from Greek'''
    # Inherits from the Translation class
    def __init__(self, source_language, target_language):
        super().__init__(source_language, target_language)

    def translate_document(self, file):
        '''Function to translate a given document from Greek to English and extract key words'''
        
        logger.info("Starting document translation...")
        
        document_type = os.path.splitext(file.name)[1][1:]
        logger.info(f"Document type: {document_type}")
        
        text = self.open_pdf(file.name) if 'pdf' in document_type else self.open_word(file.name)
        logger.info(f"Document text: {text}")
        
        translation, key_words = super().translate(text)
        
        return translation, key_words
        
    def open_pdf(self, pdf_path):
        '''Opens a pdf file and extracts the text'''
        
        pdf = PdfReader(pdf_path)
        text = ""
        for page in range(len(pdf.pages)):
            text += pdf.pages[page].extract_text()
        
        return text
           
    def open_word(self, word_path):
        '''Opens a word file and extracts the text'''
        
        document = Document(word_path)
        text = ""
        for para in document.paragraphs:
            text += para.text
        
        return text
    
    # def chunk_text(self, text):
    #     '''Chunks the text into smaller pieces'''
    #     pass
    
       

In [55]:
document_translator = DocumentTranslation("Greek", "English")

file = "new3_1.pdf"
file_2 = "new3_1.docx"

translated_pdf,_ = document_translator.translate_document(file)
translated_doc,_ = document_translator.translate_document(file_2)

KeyboardInterrupt: 

In [52]:
print(translated_pdf)
print(translated_doc)

"The Thessaloniki Metro 
The construction of Thessaloniki's underground railway, that is, the Metro, has been under discussion since 1918. After many attempts, Thessaloniki's Metro started to become a reality, and it is estimated to be ready by 2023. The final cost of building the Metro would be much higher than initially calculated. There were many delays due mainly to significant archaeological findings during the Metro's construction. A whole ancient city was discovered under Thessaloniki! The Metro will culturally transform the city. It will connect the present with the past, providing the residents an opportunity to take pride in their city's history. 

Thessalonians, for many years, have suffered greatly from the Metro construction works because they have hampered their daily mobility, and business owners have lost incomes because people's access to the shops has been difficult. However, the Metro will provide a solution to the city's significant traffic problem. Distances will b

In [60]:
source_language = "English"  # Default language
target_language = "Greek" if source_language == "English" else "English"
translator = Translation(source_language, target_language)
transcriber = Transcription("Greek", "English")
document_translator = DocumentTranslation("Greek", "English")
def update_language(widget, state):
    global source_language, target_language, translator
    source_language = widget
    target_language = "Greek" if source_language == "English" else "English"
    translator = Translation(source_language, target_language)
    return widget

def main():
       
    # Translate tab
    with gr.Blocks() as chatbot_demo:
        
        # Create a state to hold some global variable    
        source_language_state = gr.State(False)
        source_language_widget = gr.Radio(["English", "Greek"], label="Source Language",value="English")
        source_language_widget.change(update_language, inputs=[source_language_widget, source_language_state], outputs=[source_language_state])
        
    
        with gr.Row():  # Define a Row
            source_language_widget  # Add your widget within the Row

        with gr.Row():  # Define another Row
            with gr.Column():
                chatbot = gr.Chatbot([], elem_id="Translation bot", label = 'Translation bot').style(height=250)
                msg = gr.Textbox()
                clear = gr.Button("Clear")

                def respond(message, chat_history):

                    bot_message, key_words = translator.translate(message)  # Use your chatbot_completition function here
                    chat_history.append((message, bot_message))
                    return "", chat_history, key_words

                msg.submit(respond, [msg, chatbot], [msg, chatbot])
                clear.click(lambda: None, None, chatbot, queue=False)
        
                with gr.Accordion("Greek examples", open=False):
                    gr.Examples(
                        examples=["Είναι μια όμορφη ηλιόλουστη μέρα στην παραλία.",
                                    "Αυτός είναι ο καλύτερος φίλος μου",
                                    "Το παιδί έχει πολλά παιχνίδια"],
                        inputs=msg
                    )
            
            with gr.Column():
                key_words_textbox = gr.Textbox(label='Vocab',interactive= False)  # Define a textbox to show the keywords
                msg.submit(respond, [msg, chatbot], [msg, chatbot, key_words_textbox]) 
                
                    
    # with gr.Row():  # Define a Row
    #     source_language_widget  # Add your widget within the Row

    # audio_input = gr.inputs.Audio(source="upload", type="filepath")
    audio_input_microphone = gr.Microphone(source="microphone", type="filepath")
    transcribed_text = gr.Textbox(label='Transcription',interactive= False)
    translated_text = gr.Textbox(label='Translation',interactive= False)
    vocab = gr.Textbox(label='Vocab',interactive= False)
    # with gr.Row():  # Define another Row
    #     audio_input
    #     audio_input_microphone

    transcription_interface = gr.Interface(
        fn= transcriber.transcribe,
        inputs= audio_input_microphone,
        outputs=[transcribed_text, translated_text,vocab],
        title="Audio Transcription",
        description="Record audio using the microphone, Greek to English only",
        live=True
    )

    translated_doc = gr.Textbox(label='Translated Document',interactive= False)
    translated_doc_vocab = gr.Textbox(label='Vocab',interactive= False)

    document_translation_interface = gr.Interface(
        fn = document_translator.translate_document,
        inputs= gr.File(label="Document", type="file"),
        outputs= [translated_doc, translated_doc_vocab],
        title="Document Translation",
        description="Translate a document from Greek to English",
        live=True
    )
    

    # chatbot_demo.launch(share=True)
    tabbed_interface = gr.TabbedInterface([chatbot_demo,transcription_interface, document_translation_interface], ["Translate", "Transcribe", "Document Translation"])

    tabbed_interface.launch(share=True)

if __name__ == "__main__":
    main()
    
    

c:\Python311\Lib\site-packages\gradio\components\chatbot.py:228: UserWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  warnings.warn(
c:\Python311\Lib\site-packages\gradio\blocks.py:922: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Running on local URL:  http://127.0.0.1:7895
Running on public URL: https://cd9145119419eca815.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [37]:
def transcribe(audio_file):
    transcription, translation = transcriber.transcribe(audio_file)
    
    return transcription, translation
    
    
with gr.Blocks() as demo:


    audio_input_microphone = gr.Microphone(source="microphone", type="filepath")
    transcribed_text = gr.Textbox(label='Transcription',interactive= False)
    translated_text = gr.Textbox(label='Translation',interactive= False)
    vocab = gr.Textbox(label='Vocab',interactive= False)
    
    btn = gr.Button(value="Submit")
    btn.click(transcribe, inputs=audio_input_microphone, outputs=[transcribed_text,translated_text, vocab])
    gr.Markdown("Example")
    gr.Examples(
        examples=["greek1.mp3"],
        inputs= gr.Audio(source="upload", type="filepath"),
        outputs=[transcribed_text,translated_text],
        fn=transcribe,
        cache_examples=True,
    )


if __name__ == "__main__":
    demo.launch()

Caching examples at: 'C:\Users\Jordan Moshcovitis\Desktop\Coding\Chat_bots\greek_translation_transcribe\gradio_cached_examples\535'
Running on local URL:  http://127.0.0.1:7893

To create a public link, set `share=True` in `launch()`.


In [ ]:
# gradio test code
# structure of the app
# translate tab translalte and vocab check to select language

source_language = "English"  # Default language
target_language = "Greek" if source_language == "English" else "English"
translator = Translation(source_language, target_language)

def update_language(widget, state):
    global source_language, target_language, translator
    source_language = widget
    target_language = "Greek" if source_language == "English" else "English"
    translator = Translation(source_language, target_language)
    return widget

def main():
       
    # Translate tab
    with gr.Blocks() as chatbot_demo:
        # Create a state to hold some global variable    
        source_language_state = gr.State(False)
        source_language_widget = gr.Radio(["English", "Greek"], label="Source Language",value="English")
        source_language_widget.change(update_language, inputs=[source_language_widget, source_language_state], outputs=[source_language_state])
        
 
        with gr.Row():  # Define a Row
            source_language_widget  # Add your widget within the Row

        with gr.Row():  # Define another Row
            with gr.Column():
                chatbot = gr.Chatbot([], elem_id="Translation bot", label = 'Translation bot').style(height=250)
                msg = gr.Textbox()
                clear = gr.Button("Clear")

                def respond(message, chat_history):

                    bot_message, key_words = translator.translate(message)  # Use your chatbot_completition function here
                    chat_history.append((message, bot_message))
                    return "", chat_history, key_words

                msg.submit(respond, [msg, chatbot], [msg, chatbot])
                clear.click(lambda: None, None, chatbot, queue=False)
        
                with gr.Accordion("Greek examples", open=False):
                    gr.Examples(
                        examples=["Είναι μια όμορφη ηλιόλουστη μέρα στην παραλία.",
                                    "Αυτός είναι ο καλύτερος φίλος μου",
                                    "Το παιδί έχει πολλά παιχνίδια"],
                        inputs=msg
                    )
            
            with gr.Column():
                key_words_textbox = gr.Textbox(label='Vocab',interactive= False)  # Define a textbox to show the keywords
                msg.submit(respond, [msg, chatbot], [msg, chatbot, key_words_textbox]) 
    
    with gr.Blocks() as transcriber:
        # Create a state to hold some global variable    
        source_language_state = gr.State(False)
        source_language_widget = gr.Radio(["English", "Greek"], label="Source Language",value="Greek")
        source_language_widget.change(update_language, inputs=[source_language_widget, source_language_state], outputs=[source_language_state])
        
        with gr.Row():  # Define a Row
            source_language_widget  # Add your widget within the Row

        audio_input = gr.inputs.Audio(source="upload", type="filepath")
        audio_input_microphone = gr.inputs.Audio(source="microphone", type="filepath")
        transcribed_text = gr.outputs.Textbox()
        translated_text = gr.outputs.Textbox()
    
    # # Transcription tab
    # # set language pair with checkboxes
    # source_language = gr.inputs.CheckboxGroup(["English", "Greek"], label="Source Language")
    # target_language = "Greek" if source_language == "English" else "English"
    

    # # create audio input for microphone 
    # audio_input = gr.inputs.Audio(source="microphone", type="filepath")
    
    # output_text = gr.outputs.Textbox()
    
    # transcription_interface = gr.Interface(
    #     fn= transcribe_audio,
    #     inputs=audio_input,
    #     outputs=output_text,
    #     title="Audio Transcription",
    #     description="Upload an audio file or record audio using the microphone, and hit the 'Submit' button.",
    #     live=True
    # )
    
    tabbed_interface = gr.TabbedInterface([transcription_interface, chatbot_demo, chatbot_demo2], ["Transcription", "Chatbot transcription", "Chatbot translate"])

    tabbed_interface.launch(share=True)

if __name__ == "__main__":
    main()
    
    
    
    

In [34]:
transcriber = Transcription("Greek", "English")

audio_path = "greek1.mp3"

# audio = AudioSegment.from_mp3(audio_path)

proofread, translation = transcriber.transcribe(audio_path)
print(proofread)
print(translation)

"Γεια σου γιαγιά! Τι κάνεις? Καλός το παλικάρι μου! Μόλις τώρα έφτιαξα το αγαπημένο σου φαγητό, τα κουπέπια! Τι τυχερός που είμαι! Πώς τα κάνεις γιαγιά? Πρώτα απ' όλα, χρειάζεσαι αμπελόφυλλα. Όταν ήμουν μικρή, η μητέρα μου μάζευε τα αμπελόφυλλα από την κλιματαριά μας. Τότε όλα τα σπίτια είχαν κλιματαριές, ενώ τώρα αγοράζουν τα φύλλα από την αγορά. Τα περισσότερα υλικά που χρησιμοποιούσαμε παλιά στο μαγείρεμα"
"Hello grandmother! How are you? Here's my good lad! I just made your favorite dish, the stuffed vine leaves! How lucky I am! How do you do it, grandmother? First of all, you need vine leaves. When I was young, my mother used to collect vine leaves from our vineyard. Back then, all houses had vineyards, while now they buy the leaves from the market. Most of the ingredients we used to use in our cooking were gathered this way."


In [7]:
class Transcription:
    def __init__(self, source_language, target_language):
        self.source_language = source_language
        self.target_language = target_language
        self.model = whisper.load_model("large")
        self.translate = Translation(source_language, target_language)

    def transcribe(self, audio_file):
        '''Transcribes the audio file and proofreads the transcription'''

        ### Size (in bytes) = (Sample Rate) * (Bit Depth/8) * (Number of Channels) * (Duration in seconds)
        ### Size = 44,100 * (16/8) * 1 * 60 = 5,292,000 bytes, which is approximately 5 MB.

        # Calculate the size of the audio file
        audio_file_size = os.path.getsize(audio_file)
        audio_length = audio_file_size / ( 44100 * (16/8) * 1 )
        logger.info(f"Audio file size: {audio_file_size} bytes and {audio_length} seconds")

        # Transcribe the audio
        logger.info("Starting transcription...")
        transcription = self.transcribe_audio(audio_file)
        logger.info(f"Transcription completed: {transcription}")

        # Proofread the transcription
        logger.info("Proofreading transcription...")
        proofread_transcription = self.proofread_transcription(transcription)
        logger.info(f"Proofread transcription: {proofread_transcription}")

        # Translate the transcription
        # translation,key_words = self.translate.translate(proofread_transcription)
        
        return proofread_transcription, 
    def transcribe_audio(self, audio_path):
        '''Transcribes an audio file using the whisper model'''
        
        # Check if file size is less than 25 MB
        file_size = os.path.getsize(audio_path) / (1024 * 1024) # size in MB
        if file_size > 25:
            raise ValueError(f"The file size is {file_size:.2f} MB which exceeds the 25 MB limit.")

        # Check if file extension is supported
        supported_file_types = ['mp3', 'mp4', 'mpeg', 'mpga', 'm4a', 'wav', 'webm']
        file_extension = os.path.splitext(audio_path)[1][1:]
        if file_extension not in supported_file_types:
            raise ValueError(f"The file type '{file_extension}' is not supported. Please use one of the following: {', '.join(supported_file_types)}.")

        # If checks pass, transcribe the audio
        audio_file = open(audio_path, "rb")
        transcript = openai.Audio.transcribe("whisper-1", audio_file)

        # Always close the file you opened
        audio_file.close()

        return transcript
        
        # result = self.model.transcribe(audio_file)        
        # return result["text"]

    def proofread_transcription(self, transcription):
        '''Improves the transcription quality using GPT-4'''
        
        SYSTEM = """You are an expert {self.source_langauge} transcription agent. In particular, an advanced language model trained to proofread and correct text based on grammar, syntax, and coherence in context. Your role is to carefully examine the given text in {self.source_langauge}, identify semantic errors resulting from transcription, and then correct these errors. Use your knowledge of {self.source_language} language and syntax to ensure the accuracy and coherence of the transcriptions. Your task is crucial for the quality and usefulness of the transcriptions. You will be given {self.source_langauge} text that has been transcribed but may contain errors. Your job is to fix these errors to the best of your ability. Respond using markdown.
        
        You will work through proofreading and correcting the transcription in the following way step by step:
        
        Identify the context: [context]
        Identify key words: [key words]
        Correct the transcription: [transcription]
        
        You will not add any additional detail to the transcription. You will only correct the transcription. You will only return the transcription. You will not return the context or key words.
        
        Write the transcription here:
               
        
        """
        
        PROMPT = f"Please proofread and correct this transcription: \"{transcription}\""
        proofread_transcription = chatbot_completition(SYSTEM, PROMPT)
        return proofread_transcription

In [8]:

transcriber = Transcription("English", "English")

# audio_path = "greek1.mp3"
audio_path = "what1.mp3"
# audio = AudioSegment.from_mp3(audio_path)

proofread = transcriber.transcribe(audio_path)
print(proofread)


('"Let\'s get into the plan\'s details. There are several areas I wanted to discuss. Initially, we\'ll examine what I\'m sending you, starting with the client profile that contains a link to your plan and a check-in sheet. The check-in sheet is for facilitating the process of checking in – its main aim is providing me with all the information I need. Some clients are diligent enough and do not require the check-in sheet, but often, I recommend its use. I pay keen attention to aspects such as blood pressure, blood glucose, sleep, and your adherence to the plan. Your mental response to the protocol is also paramount.\n\nYour plan profile contains the plan and the check-in sheet. The client plan details your objectives: nootropic aid, improving hormone profile, and libido; reducing refractory period, creating safe muscle gain protocols, and addressing existing issues like erectile dysfunction, mild depression, and hypogonadism. You didn\'t request dietary help, but if necessary, I can pro

In [10]:
audio_path = "what2.mp3"
# audio = AudioSegment.from_mp3(audio_path)

proofread = transcriber.transcribe(audio_path)
print(proofread)

('"So, I would say, for HCG, what I would model would be that I really would prefer to use testosterone at a TRT dose and have HCG in the background to guarantee that adequate neurosteroid cascade kind of signaling. I\'d introduce it and withdraw it probably two months on, two months off, just to minimize the risk of desensitization, even though it\'s not explicitly clear in the literature. Also, I believe HCG is immensely valuable for men on TRT because it helps to maintain fertility and to keep neurosteroids active, which are important. When you inject testosterone versus when your body produces it via the LH or FSH pulse, it\'s completely different because when you inject it, you\'re bypassing a lot of the natural processes. So, yes, it\'s possible. That\'s why I prefer a two-month on and one-month off approach. We can certainly discuss more about this too. If you\'d like, we can arrange another call, but I\'m going to send the plan right now."',)
